# Assigment - 2 - Web Scrapping
By Mónica Atiaga

Batch - DSNB1222

1. All the questions must be done in a single Jupyternotebook.
2. There should be proper comments in code.

In [1]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
import time

In [8]:
#!pip install webdriver-manager

**Q1:** Write a python program to scrape data for “Data Analyst” Job position in “Bangalore” location. You 
have to scrape the job-title, job-location, company_name, experience_required. You have to scrape first 10 
jobs data.
This task will be done in following steps:

1. First get the webpage https://www.naukri.com/
2. Enter “Data Analyst” in “Skill, Designations, Companies” field and enter “Bangalore” in “enter the 
location” field.
3. Then click the searchbutton.
4. Then scrape the data for the first 10 jobs results youget.
5. Finally create a dataframe of the scraped data.
**Note: All of the above steps have to be done in code. No step is to be done manuall**

In [3]:
driver = webdriver.Chrome()
driver.get("http://www.naukri.com/")

designation = driver.find_element(By.CLASS_NAME, "suggestor-input")
designation.send_keys('Data Analyst')

location = driver.find_element(By.XPATH, "/html/body/div/div[7]/div/div/div[5]/div/div/div/div[1]/div/input")
location.send_keys('Bangalore')

search = driver.find_element(By.CLASS_NAME,"qsbSubmit")
search.click()

**Q2:** Write a python program to scrape data for “Data Scientist” Job position in “Bangalore” location. You 
have to scrape the job-title, job-location, company_name. You have to scrape first 10 jobs data.
This task will be done in following steps:
1. First get the webpage https://www.naukri.com/
2. Enter “Data Scientist” in “Skill, Designations, Companies” field and enter “Bangalore” in “enter the location” field.
3. Then click the searchbutton.
4. Then scrape the data for the first 10 jobs results youget.
5. Finally create a dataframe of the scraped data.
**Note: All of the above steps have to be done in code. No step is to be done manually**

**Q3:** In this question you have to scrape data using the filters available on the webpage as shown below

You have to use the location and salary filter.

You have to scrape data for “Data Scientist” designation for first 10 job results.

You have to scrape the job-title, job-location, company name, experience required. 

The location filter to be used is “Delhi/NCR”. The salary filter to be used is “3-6” lakhs

The task will be done as shown in the below steps:

1. first get thewebpage https://www.naukri.com/
2. Enter “Data Scientist” in “Skill, Designations, and Companies” field.
3. Then click the searchbutton.
4. Then apply the location filter and salary filter by checking the respectiveboxes
5. Then scrape the data for the first 10 jobs results youget.
6. Finally create a dataframe of the scrapeddata.
**Note: All of the above steps have to be done in code. No step is to be done manually.**

**Q4:** Scrape data of first 100 sunglasses listings on flipkart.com. You have to scrape four attributes:
1. Brand
2. Product Description
3. Price

The attributes which you have to scrape is ticked marked in the below image

To scrape the data you have to go through following steps:
1. Go to Flipkart webpage by url :https://www.flipkart.com/
2. Enter “sunglasses” in the search field where “search for products, brands and more” is written and click the search icon
3. After that you will reach to the page having a lot of sunglasses. From this page you can scrap the required data as usual.
4. After scraping data from the first page, go to the “Next” Button at the bottom other page , then click on it.
5. Now scrap data from this page as usual
6. Repeat this until you get data for 100 sunglasses.
**Note: That all of the above steps have to be done by coding only and not manually.**


In [18]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import regex as re
import math
import random


In [46]:
def load_products_by_page(driver, n=None):
    ''' Load the products with brand, description and price on the page.
        n: number of products required
        Return: a DataFrame with the info of the products
    '''
    # scraping Brands using list comprehension
    brand_tags = driver.find_elements(By.XPATH, '//div[@class="_2WkVRV"]')
    brand = [ tag.text for tag in brand_tags]
    print("Brands retrived:", len(brand))
    
    # scraping Product Descriptions: some products  class="IRpwTa" and other class="IRpwTa _2-ICcC"
    prod_tags = driver.find_elements(By.XPATH,'//a[@class="IRpwTa" or @class="IRpwTa _2-ICcC"]') 
    product_desc = [ tag.text for tag in prod_tags]
    print("Desc. retrived:", len(product_desc))

    
    # scraping Price
    price_tags = driver.find_elements(By.XPATH, '//div[@class="_30jeq3"]')
    price = [tag.text for tag in price_tags]
    print("Prices retrived:", len(price))
    
    # % off 
    poff_tags = driver.find_elements(By.XPATH, '//div[@class="_3Ay6Sb"]')
    p_off = [tag.text for tag in poff_tags]
    print("% off retived:", len(p_off))
    
     
    if(len(brand) == len(product_desc) and len(brand) == len(price) and len(brand)==len(p_off)):
        # n=None or the required records are equal or more than the retrived records
        if (not(n) or len(brand)<=n):
            return pd.DataFrame({'Brand': brand,
                              'Product description': product_desc,
                              'Price': price,
                              '% off': p_off})
        elif(len(brand)> n):
            return pd.DataFrame({'Brand': brand[:n],
                              'Product description': product_desc[:n],
                              'Price': price[:n],
                              '% off': p_off[:n]})
    
    return pd.DataFrame()

def go_next_page(driver, i):
    ''' Go to the next page of products'''
    next_btn = driver.find_elements(By.XPATH, '//a[@class="ge-49M"]')
    
    # scroll until button be visible
    actions = ActionChains(driver)
    actions.move_to_element(next_btn[i]).perform()

    # Wait until the button be visible (loading compleated)
    wait = WebDriverWait(driver, 30)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@class="ge-49M"]')))

    # Click the button once it is clickable
    next_btn[i].click()
    

def wait_random(driver):
    '''Simulate random number of timeouts'''
    time_to_wait = random.choice([60, 15, 30, 45])
    print(f"Waiting..{time_to_wait} seconds.")
    WebDriverWait(driver, time_to_wait)   # Esperar hasta n segundos
    
def retrive_product_per_page(driver):
    # Retrive the total number of products from the search and 
    # the number of products on the retrived page
    reg = driver.find_element(By.XPATH, '//span[@class="_10Ermr"]')

    pattern = r'\b\d{1,3}(?:,\d{3})*\b|\b\d+\b'
    matches = re.findall(pattern, reg.text)
    numbers_without_commas = [match.replace(',', '') for match in matches]
    reg_1st, reg_2d, tot_reg = [*map(int, numbers_without_commas)]
    
    return [reg_1st, 
            reg_2d, 
            tot_reg]

In [89]:
def cleaning_products(df):
    # Cleaning the data
    pattern = r'(\d+%)'
    df['% off'] = df['% off'].str.extract(pattern)

    # Reseting index
    df.reset_index(drop=True, inplace=True)
    
    return (df)
    
def remove_popup():
    # Check if an element on the new page is present
    try:
        close_button  = driver.find_element(By.XPATH, '//button[@class="_2KpZ6l _2doB4z"]')
        close_button.click() 
        print("Pop-up removed")
    except NoSuchElementException:
        print("No pop-up windows was found")


def retrieve_products(n):
    ''' Retrieve the products advancing page by page until completing all "n" products.
        Return: a dataframe with the products'''
    # Checking the found records
    first, second, total = retrive_product_per_page(driver)
    total_records_to_retrive = n

    if(total < total_records_to_retrive):
        total_records_to_retrive = total

    df_products = pd.DataFrame()
    i = 0 

    while( total_records_to_retrive > 0 ):

        print('*'*40, 'i=' ,i)

        wait_time = 10
        time.sleep(wait_time)

        df = load_products_by_page(driver, total_records_to_retrive) 
        df_products = pd.concat([df_products,df])

        total_records_to_retrive -= len(df)

        print("total_records_to_retrive left", total_records_to_retrive)
        print("retrived records:", len(df_products))

        if (total_records_to_retrive > 0):
            wait_random(driver)
            go_next_page(driver, i)

        i += 1

    print( f'Products retrived successfully: {len(df_products)}' )
    return (df_products)
        
        
def search_product(product):
    try:
        input = driver.find_element(By.XPATH, '//input[@class="Pke_EE" or @class="_3704LK"]') 
        input.send_keys(product)
        print(f"Search for {product}")

        WebDriverWait(driver, 10)

        btn = driver.find_element(By.XPATH, '//button[@class="_2iLD__" or @class="L0Z3Pu"]') 
        btn.click()
        print("Click successful")

    except NoSuchElementException:
        print("An element wasn't found.")
        


In [83]:
driver = webdriver.Chrome()

In [84]:
driver.get("https://www.flipkart.com/")

# Closing pop-up window
wait = WebDriverWait(driver, 30)

remove_popup()

Pop-up removed


In [85]:
product = 'sunglasses'
search_product(product)

Search for sunglasses
Click successful


In [90]:
df_products = retrieve_products(100)

**************************************** i= 0
Brands retrived: 40
Desc. retrived: 40
Prices retrived: 40
% off retived: 40
total_records_to_retrive left 60
retrived records: 40
Waiting..30 seconds.
**************************************** i= 1
Brands retrived: 40
Desc. retrived: 40
Prices retrived: 40
% off retived: 39
total_records_to_retrive left 60
retrived records: 40
Waiting..45 seconds.
**************************************** i= 2
Brands retrived: 40
Desc. retrived: 40
Prices retrived: 40
% off retived: 40
total_records_to_retrive left 20
retrived records: 80
Waiting..15 seconds.
**************************************** i= 3
Brands retrived: 40
Desc. retrived: 40
Prices retrived: 40
% off retived: 40
total_records_to_retrive left 0
retrived records: 100
Products retrived successfully: 100


In [94]:
# Cecking the size of dataframe
df_products.shape

(100, 4)

In [95]:
df_products = cleaning_products(df_products)    
display(df_products.tail())

# Saving the result dataset
df_products.to_csv('webScraping2_datasets/q4_products.csv')

,Brand,Product description,Price,% off
95,Rich Club,Polarized Round Sunglasses (48),₹193,61%
96,CHORIOTIS,"UV Protection, Riding Glasses, Mirrored Round ...","₹1,999",25%
97,IDEE,Gradient Butterfly Sunglasses (64),"₹2,275",30%
98,SUNBEE,UV Protection Cat-eye Sunglasses (Free Size),₹199,66%
99,PIRASO,UV Protection Wayfarer Sunglasses (32),₹259,83%


In [96]:
# Cleaning
driver.quit()

**Q5:** Scrape 100 reviews data from flipkart.com for iphone11 phone. You have to go the link: 
https://www.flipkart.com/apple-iphone-11-black-64-gb/product-reviews/itm4e5041ba101fd?pid=MOBFWQ6BXGJCEYNY&lid=LSTMOBFWQ6BXGJCEYNYZXSHRJ&marketplace=FLIPKART

As shown in the above page you have to scrape the tick marked attributes. These are:
1. Rating
2. Review summary
3. Full review
4. You have to scrape this data for first 100reviews.
**Note: All the steps required during scraping should be done through code only and not manually.**

**Q6:** Scrape data forfirst 100 sneakers you find when you visit flipkart.com and search for “sneakers” in the 
search field.

You have to scrape 3 attributes of each sneaker:
1. Brand
2. ProductDescription
3. Price

As shown in the below image, you have to scrape the above attributes.

In [97]:
driver = webdriver.Chrome()

In [98]:
driver.get("https://www.flipkart.com/")

# Closing pop-up window
wait = WebDriverWait(driver, 30)

remove_popup()

Pop-up removed


In [99]:
product = 'sneakers'
search_product(product)

Search for sneakers
Click successful


In [ ]:
df_products = retrieve_products(100)

**************************************** i= 0
Brands retrived: 40
Desc. retrived: 40
Prices retrived: 40
% off retived: 40
total_records_to_retrive left 60
retrived records: 40
Waiting..60 seconds.
**************************************** i= 1


In [67]:
df_products.shape

,Brand,Product description,Price,% off
0,Airson,Junior Zero1 Sports shoes for Men | Gym Traini...,₹989,55% off
1,Sparx,Stylish and Comfirtable Shoes For Mens Sneaker...,₹749,16% off
2,aadi,Synthetic Leather |Lightweight|Comfort|Summer|...,₹399,80% off
3,Nobelite,Sneakers For Men,₹289,71% off
4,aadi,Sneakers For Men,₹299,85% off
...,...,...,...,...
15,asian,"SM-162 Black Walking ,Training,Sneakers,Loafer...",₹499,28% off
16,RapidBox,Sneakers For Men,₹559,44% off
17,Abros,Flex Sneakers For Men,"₹1,521",30% off
18,PUMA,Hustle V2 Sneakers For Men,"₹1,599",60% off


In [68]:
# Cleaning the products and set index
cleaning_products(df_products)    
display(df_products.tail())

# Saving the result dataset
df_products.to_csv('webScraping2_datasets/q6_sneakers.csv')

,Brand,Product description,Price,% off
95,asian,"SM-162 Black Walking ,Training,Sneakers,Loafer...",₹499,28%
96,RapidBox,Sneakers For Men,₹559,44%
97,Abros,Flex Sneakers For Men,"₹1,521",30%
98,PUMA,Hustle V2 Sneakers For Men,"₹1,599",60%
99,Kraasa,Sneakers For Women,₹399,60%


In [69]:
# Cleaning
driver.quit()


**Q7:** Go to webpage https://www.amazon.in/ Enter “Laptop” in the search field and then click the search icon. Then 
set CPU Type filter to “Intel Core i7” as shown in the below image:

After setting the filters scrape first 10 laptops data. You have to scrape 3 attributes for each laptop:
1. Title
2. Ratings
3. Price

In [14]:
driver = webdriver.Chrome()

In [15]:
driver.get("https://www.amazon.in/") 

In [20]:
product = 'Laptop'

input = driver.find_element(By.XPATH, '//input[@id="twotabsearchtextbox"]') 
input.send_keys(product)

wait_random(driver)

btn = driver.find_element(By.XPATH, '//input[@id="nav-search-submit-button"]') 
btn.click()

Waiting..140 seconds.


**Q8:** Write a python program to scrape data for Top 1000 Quotes of All Time.
The above task will be done in following steps:
1. First get the webpage https://www.azquotes.com/
2. Click on TopQuotes
3. Than scrap a) Quote b) Author c) Type Of Quotes


**Q9:** Write a python program to display list of respected former Prime Ministers of India(i.e. Name, Born-Dead, 
Term of office, Remarks) from https://www.jagranjosh.com/.

This task will be done in following steps:
1. First get the webpagehttps://www.jagranjosh.com/
2. Then You have to click on the GK option
3. Then click on the List of all Prime Ministers of India
4. Then scrap the mentioned data and make theDataFrame.


**Q10:** Write a python program to display list of 50 Most expensive cars in the world (i.e. 
Car name and Price) from https://www.motor1.com/

This task will be done in following steps:
1. First get the webpagehttps://www.motor1.com/
2. Then You have to type in the search bar ’50 most expensive cars’
3. Then click on 50 most expensive carsin the world..
4. Then scrap the mentioned data and make the dataframe.